# Directive Engines

In this notebook, we'll explore how to use directive engines to orchestrate multiple directives and build complex queries. Engines provide a way to organize and combine directives in a structured, maintainable way.

## Introduction to Directive Engines

While individual directives are powerful, real-world search applications often require combining multiple query conditions. Directive engines solve this problem by:

1. Organizing directives in a structured way
2. Handling the coordination between directives
3. Managing value mapping and parameter resolution
4. Building the final query structure

The toolkit provides two main types of engines:

- **DirectiveEngine**: For standard boolean queries
- **FunctionScoreEngine**: For queries that need custom scoring functions

## Setup

Let's import the necessary modules:

In [ ]:
import json
from elastictoolkit.queryutils.builder.directiveengine import DirectiveEngine
from elastictoolkit.queryutils.builder.matchdirective import (
    ConstMatchDirective,
    TextMatchDirective,
    RangeMatchDirective,
    FieldExistsDirective
)
from elastictoolkit.queryutils.builder.directivevaluemapper import DirectiveValueMapper
from elastictoolkit.queryutils.types import FieldValue
from elastictoolkit.queryutils.consts import FieldMatchType, MatchMode

# Helper function to print queries as formatted JSON
def print_query(engine):
    query = engine.to_dsl()
    print(json.dumps(query.to_query(), indent=2))

## Creating a Simple Engine

Let's start by creating a simple directive engine for product search:

In [ ]:
class ProductSearchEngine(DirectiveEngine):
    # Define directives as class attributes
    category = ConstMatchDirective(rule=FieldMatchType.ANY)
    in_stock = ConstMatchDirective(rule=FieldMatchType.ANY)
    
    class Config:
        # Engine configuration
        value_mapper = None  # Will be set later

# Create a value mapper
class ProductValueMapper(DirectiveValueMapper):
    category = FieldValue(
        fields=["category"],
        values_list=["match_params.category"]
    )
    in_stock = FieldValue(
        fields=["in_stock"],
        values_list=[True]
    )

# Set the value mapper | This is for demo purpose only - Always create ValueMapper and directly set it in the Config of a DirectiveEngine
ProductSearchEngine.Config.value_mapper = ProductValueMapper()

# Create an engine instance with match parameters
engine = ProductSearchEngine().set_match_params({"category": "electronics"})

# Generate the query
print_query(engine)

This simple engine generates a boolean query that matches documents in the "electronics" category that are in stock. Let's break down how it works:

1. We define a `ProductSearchEngine` class that inherits from `DirectiveEngine`
2. We define directives as class attributes (`category` and `in_stock`)
3. We create a `ProductValueMapper` to map directive names to fields and values
4. We set the value mapper in the engine's `Config` class
5. We create an engine instance with match parameters
6. We generate the query using the `to_dsl()` method

The engine automatically discovers the directives, applies the value mapping, and builds the query.

## Adding More Directives

Let's expand our engine with more directives for a more comprehensive product search:

In [ ]:
class AdvancedProductValueMapper(DirectiveValueMapper):
    category = FieldValue(
        fields=["category"],
        values_list=["match_params.category"]
    )
    in_stock = FieldValue(
        fields=["in_stock"],
        values_list=[True]
    )
    price_range = FieldValue(
        fields=["price"],
        values_map={
            "gte": "match_params.min_price",
            "lte": "match_params.max_price"
        }
    )
    search = FieldValue(
        fields=["name", "description"],
        values_list=["match_params.search_text"]
    )

class AdvancedProductSearchEngine(DirectiveEngine):
    # Basic filters
    category = ConstMatchDirective(rule=FieldMatchType.ANY)
    in_stock = ConstMatchDirective(rule=FieldMatchType.ANY)
    
    # Price range
    price_range = RangeMatchDirective()
    
    # Text search
    search = TextMatchDirective(rule=FieldMatchType.ANY)
    
    class Config:
        value_mapper = AdvancedProductValueMapper()

# Create an engine instance with match parameters
engine = AdvancedProductSearchEngine().set_match_params(
    {
        "category": "electronics",
        "min_price": 50,
        "max_price": 200,
        "search_text": "wireless headphones"
    }
)

# Generate the query
print_query(engine)

This advanced engine generates a more complex query that:

1. Matches documents in the "electronics" category
2. Ensures they are in stock
3. Filters by price range (50-200)
4. Performs a text search for "wireless headphones" in the name and description fields

All of this is achieved with a clean, declarative syntax that separates the query structure from the runtime parameters.

## Conditional Directives

In many cases, you want to apply certain directives only if specific parameters are provided. Let's see how to handle this:

In [ ]:
class ConditionalValueMapper(DirectiveValueMapper):
    in_stock = FieldValue(
        fields=["in_stock"],
        values_list=[True]
    )
    category = FieldValue(
        fields=["category"],
        values_list=["match_params.category"]
    )
    brand = FieldValue(
        fields=["brand"],
        values_list=["match_params.brand"]
    )
    price_range = FieldValue(
        fields=["price"],
        values_map={
            "gte": "match_params.min_price",
            "lte": "match_params.max_price"
        }
    )
    search = FieldValue(
        fields=["name", "description"],
        values_list=["match_params.search_text"]
    )

class ConditionalSearchEngine(DirectiveEngine):
    # Always applied
    in_stock = ConstMatchDirective(rule=FieldMatchType.ANY)
    
    # Conditionally applied | if value is present in `match_params`
    category = ConstMatchDirective(rule=FieldMatchType.ANY, nullable_value=True)
    brand = ConstMatchDirective(rule=FieldMatchType.ANY, nullable_value=True)
    price_range = RangeMatchDirective(nullable_value=True)
    search = TextMatchDirective(rule=FieldMatchType.ANY, nullable_value=True)
    
    class Config:
        value_mapper = ConditionalValueMapper()

# Create an engine instance with only some parameters
engine = ConditionalSearchEngine().set_match_params(
    {
        "category": "electronics",
        "search_text": "wireless headphones"
        # No brand or price range parameters
    }
)

# Generate the query
print_query(engine)

In this example, the `brand` and `price_range` directives are not applied because the corresponding parameters are not provided. Both the directive had the parameter: `nullable_value` set to `True` and hence the engine skipped without adding any query for the respective directives. In case, this parameter (`nullable_value`) was not set, the engine would throw Exception.

## Engine Configuration

Directive engines can be configured through the `Config` class. Let's explore some configuration options:

In [ ]:
from elastictoolkit.queryutils.consts import AndQueryOp, BaseMatchOp
from elastictoolkit.queryutils.builder.helpers.valueparser import (
    RuntimeValueParser,
)

class ConfigurableEngine(DirectiveEngine):
    category = ConstMatchDirective(rule=FieldMatchType.ANY)
    search = TextMatchDirective(rule=FieldMatchType.ANY)
    
    class Config:
        value_mapper = ConditionalValueMapper()
        
        # The following are the Default Parameters
        match_directive_config = { 
            "value_parser_config": {                 # Dynamic Value Resolver Config
                "parser_cls": RuntimeValueParser,
                "prefix": "match_params",
            },
            "and_query_op": AndQueryOp.FILTER,       # Whether to use FILTER/MUST for AND query operation
            "base_match_op": BaseMatchOp.AND,        # Base Boolean condition for merging directive queries
        }

# Create an engine instance
engine = ConfigurableEngine().set_match_params(
    {
        "category": "electronics",
        "search_text": "wireless headphones"
    }
)

# Generate the query
print(":: BEFORE ::\n")
print_query(engine)


# Update the MatchDirective Config

class UpdatedConfigurationEngine(ConfigurableEngine):
    class Config:
        value_mapper = ConditionalValueMapper()
        
        # Configure how match directives are processed 
        match_directive_config = {
            "base_match_op": BaseMatchOp.OR, # This will change the base boolean condition of merging the directives which is AND by default
        }

# Create an engine instance
engine = UpdatedConfigurationEngine().set_match_params(
    {
        "category": "electronics",
        "search_text": "wireless headphones"
    }
)

# Generate the query
print("\n:: AFTER [Same Engine with Updated Configuration] ::\n")
print_query(engine)

The `match_directive_config` option allows you to configure how match directives are processed. In this example, we're configuring the value parser to allow None values and resolve list values.

## Extending Engines

You can extend existing engines to add or override directives:

In [ ]:
class BaseProductEngine(DirectiveEngine):
    category = ConstMatchDirective(rule=FieldMatchType.ANY)
    in_stock = ConstMatchDirective(rule=FieldMatchType.ANY)
    
    class Config:
        value_mapper = ConditionalValueMapper()

class ExtendedProductEngine(BaseProductEngine):
    # Add new directives
    brand = ConstMatchDirective(rule=FieldMatchType.ANY)
    search = TextMatchDirective(rule=FieldMatchType.ANY)
    
    # Override existing directives
    category = ConstMatchDirective(rule=FieldMatchType.ALL)  # Changed from ANY to ALL

# Create an engine instance
engine = ExtendedProductEngine().set_match_params(
    {
        "category": ["electronics", "audio"],  # Now requires ALL categories
        "brand": "Sony",
        "search_text": "wireless headphones"
    }
)

# Generate the query
print_query(engine)

In this example, we've extended the `BaseProductEngine` to add new directives (`brand` and `search`) and override an existing directive (`category`). This allows you to create a hierarchy of engines for different use cases while reusing common directives.

### Removing Parent Directive

You can also remove a directive from parent engine altogether by setting it to `None`

In [ ]:
class ExtendedAnyCategoryProductEngine(BaseProductEngine):
    # Add new directives
    brand = ConstMatchDirective(rule=FieldMatchType.ANY)
    search = TextMatchDirective(rule=FieldMatchType.ANY)
    
    # Removing the parent directive altogether by setting it to None
    category = None

# Create an engine instance
engine = ExtendedProductEngine().set_match_params(
    {
        "brand": "Sony",
        "search_text": "wireless headphones"
    }
)

# Generate the query
print_query(engine)

## Dynamic Engine Creation

In very rare cases, you might want to create engines dynamically based on runtime conditions. Let's see how to do this:

__NOTE__: The __prefered way__ of solving the usecase demonstrated below would be setting the values in `match_params` to `None` and setting the `nullable_value` parameter for `price_range` and `brand` directives to `True` instead of generating Engine dynamically. Use the following for very advanced usecases only.

In [ ]:
def create_dynamic_engine(include_price=False, include_brand=False):
    class DynamicEngine(DirectiveEngine):
        category = ConstMatchDirective(rule=FieldMatchType.ANY)
        in_stock = ConstMatchDirective(rule=FieldMatchType.ANY)
        search = TextMatchDirective(rule=FieldMatchType.ANY)
        
        class Config:
            value_mapper = ConditionalValueMapper()
    
    # Conditionally add directives
    if include_price:
        DynamicEngine.price_range = RangeMatchDirective()
    
    if include_brand:
        DynamicEngine.brand = ConstMatchDirective(rule=FieldMatchType.ANY)
    
    return DynamicEngine

# Create a dynamic engine with price range but no brand
DynamicEngine = create_dynamic_engine(include_price=False, include_brand=False)

# Create an engine instance
engine = DynamicEngine().set_match_params(
    {
        "category": "electronics",
        "search_text": "wireless headphones",
        "min_price": 50,
        "max_price": 200
    }
)

# Generate the query
print_query(engine)

This approach allows you to create engines with different sets of directives based on runtime conditions. It's particularly useful for building flexible search interfaces where the available filters might change based on user permissions, context, or other factors.

## Handling Errors

When working with directive engines, you might encounter errors if directives are misconfigured or if required parameters are missing. Let's see how to handle these errors:

In [ ]:
class ErrorHandlingEngine(DirectiveEngine):
    in_stock = ConstMatchDirective(rule=FieldMatchType.ANY)
    category = ConstMatchDirective(rule=FieldMatchType.ANY)
    
    class Config:
        value_mapper = ConditionalValueMapper()

# Create an engine instance without the required parameter
engine = ErrorHandlingEngine().set_match_params(
    {}  # Missing 'category' parameter
)

# Try to generate the query
try:
    query = engine.to_dsl()
    print(json.dumps(query.to_query(), indent=2))
except Exception as e:
    print(f"Error: {e}")

To handle these errors gracefully, you can use a try-except block or configure the engine to allow None values:

In [ ]:
class NullableEngine(DirectiveEngine):
    in_stock = ConstMatchDirective(rule=FieldMatchType.ANY)
    category = ConstMatchDirective(rule=FieldMatchType.ANY, nullable_value=True)
    
    class Config:
        value_mapper = ConditionalValueMapper()


# Create an engine instance without the required parameter
engine = NullableEngine().set_match_params(
    {}  # Missing 'category' parameter
)

# Generate the query
query = engine.to_dsl()
print(json.dumps(query.to_query(), indent=2))

In this case, the engine will skip the `category` directive because the required parameter is missing, but it won't raise an error.

## Summary

In this notebook, we've explored how to use directive engines to orchestrate multiple directives and build complex queries. We've covered:

- Creating simple and advanced engines
- Configuring engines with value mappers
- Handling conditional directives
- Extending engines
- Creating dynamic engines
- Handling errors

Directive engines provide a powerful way to organize and combine directives in a structured, maintainable way. They handle the coordination between directives, manage value mapping and parameter resolution, and build the final query structure.

In the next notebook, we'll explore how to use value mappers in more depth to handle complex field-value relationships.